In [10]:
from wmf import wmf 
import argparse
import textwrap
import os 
import alarmas as al
from multiprocessing import Pool
import glob
import pandas as pd 
import numpy as np 

In [11]:
rutaConfig='/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/Op_AMVA60m/configfile.md'
ListConfig=al.get_rutesList(rutaConfig)
ruta_out_rain=al.get_ruta(ListConfig, 'ruta_rain')
ruta_cuenca=al.get_ruta(ListConfig, 'ruta_cuenca')
cuenca=wmf.SimuBasin(rute = ruta_cuenca, SimSlides = True)
verbose=True

In [12]:
#cuenca
cu=cuenca
#se lee el binario de lluvia actual
rutaRain = ruta_out_rain + 'Lluvia_actual.bin'
rain_bin, rain_hdr = wmf.__Add_hdr_bin_2route__(rutaRain)
DataRain = wmf.read_rain_struct(rain_hdr)
Rain = wmf.read_mean_rain(rain_hdr)
# se lee el configfile
ListConfig=al.get_rutesList(rutaConfig)
# se leen las rutas de resultados
#rutas de salida - storage
ruta_sto = al.get_ruta(ListConfig, 'ruta_almsim')
ruta_stohist = al.get_ruta(ListConfig, 'ruta_almhist')
#rutas de salida - caudal
QsimName = al.get_ruta(ListConfig,'Qsim Name')
ruta_Qsim = al.get_ruta(ListConfig, 'ruta_qsim')
ruta_qhist = al.get_ruta(ListConfig, 'ruta_qsim_hist')
#rutas de salida - slides
ruta_out_slides = al.get_ruta(ListConfig, 'ruta_slides')
ruta_slides_bin, ruta_slides_hdr = wmf.__Add_hdr_bin_2route__(ruta_out_slides)

#Set por defecto de la modelacion
wmf.models.show_storage = 1
wmf.models.separate_fluxes = 1
wmf.models.dt = 300
wmf.models.sl_fs = 0.5
wmf.models.sim_slides = 1
posControl = wmf.models.control[wmf.models.control<>0]

#Set automatico a partir del configfile
#Param de configuracion
Lparam = ['Dt[seg]','Dx[mts]',
	'Almacenamiento medio',
	'Separar Flujos',
	'ruta_almacenamiento',
	'Retorno',
	'Simular Deslizamientos',
	'Factor de Seguridad FS',
	'Factor Corrector Zg']
DictParam = {}
for i in Lparam:
	a = al.get_ruta(ListConfig, i)
	DictParam.update({i:a})

#Prepara el tiempo y retornos
wmf.models.dt = float(DictParam['Dt[seg]'])
wmf.models.retorno = float(DictParam['Retorno'])
# Prepara los que son binarios (1) si (0) no
if DictParam['Almacenamiento medio'] == 'True':
	wmf.models.show_storage = 1
if DictParam['Separar Flujos'] == 'True':
	wmf.models.separate_fluxes = 1
if DictParam['Simular Deslizamientos'] == 'True':
	wmf.models.sim_slides = 1
	wmf.models.sl_fs = float(DictParam['Factor de Seguridad FS'])
	cu.set_Slides(wmf.models.sl_zs * float(DictParam['Factor Corrector Zg']), 'Zs')
print wmf.models.sl_zs.mean()

#Se leen las calibraciones y la configuracion de actualizar CI.
DictCalib = al.get_modelConfig_lines(ListConfig, '-c', 'Calib')
DictStore = al.get_modelConfig_lines(ListConfig, '-s', 'Store')

############################ EJECUCION ###########################################

#Prepara las ejecuciones
ListEjecs = []
Npasos = DataRain[u' Record'].shape[0]
for i in DictStore.keys():
    #trata de leer el almacenamiento 
    FileName = glob.glob(ruta_sto + DictStore[i]['Nombre'])
    if len(FileName):
        S = wmf.models.read_float_basin_ncol(ruta_sto+DictStore[i]['Nombre'],1,cu.ncells,5)[0]
    else:
        print 'Error: No se leyeron los binarios de almacenamiento operacionales.'
    #Arma la ejecucion
    Calib = DictCalib[DictStore[i]['Calib']]
    ListEjecs.append([i, Calib, rain_bin, Npasos, 1, S, ruta_sto+DictStore[i]['Nombre'],ruta_stohist+DictStore[i]['Nombre'][:-7]+'hist.json'])

4.9948


In [13]:
L=ListEjecs[0]
# def model_warper(L):
#Ejecucion del modelo
Res = cu.run_shia(L[1],L[2],L[3],L[4], 
    StorageLoc = L[5], ruta_storage=L[6], kinematicN=12,QsimDataFrame=False)
#Escribe resultados 
rutaqsim = ruta_Qsim+ QsimName +'_'+L[0].split(' ')[-1]+'.json'
Qsim = pd.DataFrame(Res['Qsim'][1:].T, 
    index=Rain.index, 
    columns=posControl)
Qsim.to_json(rutaqsim)
#Actualiza historico de caudales simulados
rutaqhist = ruta_qhist+ rutaqsim.split('/')[-1].split('.')[0]+'hist.'+rutaqsim.split('/')[-1].split('.')[-1]
al.model_write_qhist(rutaqsim,rutaqhist)
#Se actualizan los historicos de humedad de la parametrizacion asociada.
al.model_write_Stohist(L[6],L[7])
#imprime que ya ejecuto
if verbose:
    print L[0]+' ejecutado'
#     return Res

Aviso: Se ha actualizado el archivo de Qsim_historicos de: /media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/Op_AMVA60m/results/02_caudalsim/01_caudalsim_hist/Qsim_001hist.json
Aviso: Se ha actualizado el archivo de Ssim_historicos de: /media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/Op_AMVA60m/results/03_humedadsim/01_humedadsim_hist/storage_001hist.json
-s 001 ejecutado


In [14]:
pd.read_json(rutaqsim)

,1,10,100,101,102,103,104,105,106,107,...,90,91,92,93,94,95,96,97,98,99
2018-01-12 18:12:00,58.601063,0.089335,0,0.000318,25.027838,0.971680,0.042889,0.017382,49.931324,0.451066,...,0,0,0.020909,0,0.014444,50.296906,0.550220,0.000003,50.261318,0.524217
2018-01-12 18:17:00,56.489174,0.090070,0,0.000325,24.617355,1.012272,0.042897,0.017381,49.143524,0.462888,...,0,0,0.020911,0,0.014445,49.417648,0.576308,0.000003,49.387268,0.547920
2018-01-12 18:22:00,55.298534,0.090466,0,0.000331,24.373148,1.036961,0.042902,0.017380,48.671551,0.469461,...,0,0,0.020912,0,0.014447,48.900162,0.592975,0.000004,48.872414,0.562766
2018-01-12 18:27:00,54.608871,0.090670,0,0.000336,24.204716,1.050962,0.042905,0.017380,48.342159,0.472784,...,0,0,0.020912,0,0.014449,48.549915,0.602926,0.000004,48.523376,0.571453
2018-01-12 18:32:00,54.173302,0.090771,0,0.000339,24.069315,1.058270,0.042909,0.017379,48.074554,0.474229,...,0,0,0.020913,0,0.014451,48.273239,0.608459,0.000005,48.247211,0.576172
2018-01-12 18:37:00,53.858967,0.090819,0,0.000342,23.949509,1.061639,0.042912,0.017379,47.836521,0.474662,...,0,0,0.020914,0,0.014452,48.030876,0.611274,0.000008,48.005112,0.578491
2018-01-12 18:42:00,53.602013,0.090843,0,0.000344,23.839346,1.062843,0.042915,0.017378,47.617142,0.474602,...,0,0,0.020914,0,0.014454,47.808922,0.612515,0.000013,47.783287,0.579449
2018-01-12 18:47:00,53.374454,0.090854,0,0.000346,23.737257,1.062945,0.042917,0.017379,47.413788,0.474335,...,0,0,0.020915,0,0.014456,47.603371,0.612904,0.000020,47.577835,0.579686
2018-01-12 18:52:00,53.165722,0.090860,0,0.000349,23.642832,1.062556,0.042920,0.017380,47.225685,0.474010,...,0,0,0.020916,0,0.014458,47.413330,0.612871,0.000028,47.387863,0.579579
2018-01-12 18:57:00,52.972698,0.090864,0,0.000351,23.555660,1.061995,0.042922,0.017381,47.052097,0.473695,...,0,0,0.020916,0,0.014459,47.237736,0.612657,0.000036,47.212387,0.579334


In [15]:
pd.read_json(rutaqhist)

,1,10,100,101,102,103,104,105,106,107,...,90,91,92,93,94,95,96,97,98,99
2018-01-12 17:02:00,121.091942,0.070326,0.0,0.000269,41.581112,0.476218,0.041440,0.017285,82.033363,0.238086,...,0.0,0.0,0.020574,0.0,0.014430,85.109245,0.303369,0.000003,84.923653,0.275121
2018-01-12 17:07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:27:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:37:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:42:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:47:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
pd.read_json(ruta_Shist)

,Tanque 1,Tanque 2,Tanque 3,Tanque 4,Tanque 5
2018-01-12 17:07:00,1.56,0.0007,0.6130,992.27,0.07
2018-01-12 17:12:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:17:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:22:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:27:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:32:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:37:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:42:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:47:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:52:00,NaN,NaN,NaN,NaN,NaN


In [17]:
ruta_Ssim=L[6];ruta_Shist=L[7]
pd.read_csv(ruta_Ssim[:-7]+'.StOhdr', header = 4, index_col = 5, parse_dates = True, usecols=(1,2,3,4,5,6))

,Tanque 1,Tanque 2,Tanque 3,Tanque 4,Tanque 5
Fecha,,,,,
2018-01-12 18:17:00,1.23,0.0002,0.6071,992.25,0.06
2018-01-12 18:22:00,1.17,0.0002,0.6060,992.26,0.06
2018-01-12 18:27:00,1.11,0.0001,0.6050,992.29,0.06
2018-01-12 18:32:00,1.06,0.0001,0.6040,992.30,0.06
2018-01-12 18:37:00,1.01,0.0001,0.6030,992.30,0.06
2018-01-12 18:42:00,0.96,0.0001,0.6020,992.32,0.06
2018-01-12 18:47:00,0.92,0.0001,0.6010,992.32,0.05
2018-01-12 18:52:00,0.88,0.0001,0.6001,992.20,0.05
2018-01-12 18:57:00,0.83,0.0001,0.5992,992.16,0.05


In [10]:
# ruta_Ssim=L[6];ruta_Shist=L[7]
# def model_write_Stohist(ruta_Ssim,ruta_Shist):
#     ''''Actualiza el estado promedio de C.I. de cada tanque, copiandolo desde el .StOhdr a un msg antes creado.
#         -ruta_Ssim: ruta .hdr del archivo de condiciones antecedentes .StObin resultado de la simulacion.
#         -ruta_Shist: ruta .msg del archivo Shist.'''
##Se actualizan los historicos de humedad de la parametrizacion asociada.
#Lee el almacenamiento actual
Sactual = pd.read_csv(ruta_Ssim[:-7]+'.StOhdr', header = 4, index_col = 5, parse_dates = True, usecols=(1,2,3,4,5,6))
#Lee el historico
Shist = pd.read_json(ruta_Shist)
# #Actualiza
Shist=Shist.append(Sactual.iloc[[0]])
# #borra index repetidos, si los hay - la idea es que no haya pero si los hay no funciona el df.reindex
Shist=Shist.drop_duplicates()
# # Crea el index que debe tener la serie con todos los datos
rngindex=pd.date_range(Shist.index[0],Shist.index[-1],freq='5min')
# #Si hay faltantes los llena, si no deja igual el df. Esto solo funciona cuando dejamos el cron operacional y el 'freq' es exactamente 5 min.
Shist=Shist.reindex(rngindex)

# #guarda el archivo
# Shist.to_json(ruta_Shist)
# print 'Aviso: Se ha actualizado el archivo de Ssim_historicos de: '+ruta_Shist

In [36]:
Sactual

,Tanque 1,Tanque 2,Tanque 3,Tanque 4,Tanque 5
Fecha,,,,,
2018-01-12 18:07:00,1.42,0.0004,0.6105,992.28,0.06
2018-01-12 18:12:00,1.35,0.0003,0.6093,992.24,0.06
2018-01-12 18:17:00,1.29,0.0002,0.6082,992.25,0.06
2018-01-12 18:22:00,1.23,0.0002,0.6071,992.25,0.06
2018-01-12 18:27:00,1.17,0.0002,0.6060,992.26,0.06
2018-01-12 18:32:00,1.11,0.0001,0.6050,992.29,0.06
2018-01-12 18:37:00,1.06,0.0001,0.6040,992.30,0.06
2018-01-12 18:42:00,1.01,0.0001,0.6030,992.30,0.06
2018-01-12 18:47:00,0.96,0.0001,0.6020,992.32,0.06


In [11]:
Shist

,Tanque 1,Tanque 2,Tanque 3,Tanque 4,Tanque 5
2018-01-12 17:07:00,1.56,0.0007,0.6130,992.27,0.07
2018-01-12 17:12:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:17:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:22:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:27:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:32:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:37:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:42:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:47:00,NaN,NaN,NaN,NaN,NaN
2018-01-12 17:52:00,NaN,NaN,NaN,NaN,NaN


In [88]:
def model_write_qsim(rutaQsim,rutaQhist, pcont):
    ##Se actualizan los historicos de Qsim de la parametrizacion asociada.
    #Lee el almacenamiento actual
    Qactual = pd.read_json(rutaQsim)
    #Lee el historico
    Qhist = pd.read_json(rutaQhist)
    #Actualiza Qhist con Qactual.
    try:
        Qhist=Qhist.append(Qactual.iloc[[0]])#.sort_index(axis=1))
    except:
        print 'Aviso: no se esta actualizando Qhist en: '+rutaQhist
    #borra index repetidos, si los hay - la idea es que no haya pero si los hay no funciona el df.reindex
    Qhist=Qhist.drop_duplicates()
    #Crea el index que debe tener la serie con todos los datos
    rngindex=pd.date_range(Qhist.index[0],Qhist.index[-1],freq='5min')
    #Si hay faltantes los llena, si no deja igual el df. Esto solo funciona cuando dejamos el cron operacional y el 'freq' es exactamente 5 min.
    Qhist=Qhist.reindex(rngindex)
    #Guarda el archivo historico 
    Qhist.to_json(rutaQhist)
    # Aviso
    print 'Aviso: Se ha actualizado el archivo de Qsim_historicos de: '+rutaQhist

Aviso: Se ha actualizado el archivo de Qsim_historicos de: /media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/Op_AMVA60m/results/02_caudalsim/01_caudalsim_hist/Qsim_001hist.json


# Se genera una cosa y se guarda otra!!!!!

In [34]:
L=ListEjecs[0]

In [ ]:
#Ejecucion
# Cantidad de procesos 
Nprocess = len(ListEjecs)
if Nprocess > 15:
	Nprocess = int(Nprocess/1.2)
#Ejecucion  en paralelo y guarda caudales 
if args.verbose:
	print 'Resumen Ejecucion modelo'
	print '\n'
p = Pool(processes=Nprocess)
R = p.map(model_warper, ListEjecs)
p.close()
p.join()
#Un brinco para uqe quede lindo el print de deslizamientos.
if args.verbose:
	print '\n'
	print 'Resumen deslizamientos'

############################ ESCRIBE EL BINARIO DE DESLIZAMIENTOS ##################

#Archivo plano que dice cuales son las param que simularon deslizamientos 
f = open(ruta_slides_hdr,'w')
f.write('## Parametrizaciones Con Simulacion de Deslizamientos \n')
f.write('Parametrizacion \t N_celdas_Desliza \n')
#Termina de escribir el encabezado y escribe el binario.
rec = 0
for c,i in enumerate(ListEjecs):
	if DictStore[i[0]]['Slides'] == 'True':
		#Determina la cantidad de celdas que se deslizaron
		Slides = np.copy(R[c]['Slides_Map'])
		Nceldas_desliz = Slides[Slides<>0].shape[0]
		f.write('%s \t %d \n' % (i[0], Nceldas_desliz))
		#si esta verbose dice lo que pasa 
		if args.verbose:
			print 'Param '+i[0]+' tiene '+str(Nceldas_desliz)+' celdas deslizadas.'
		#Escribe en el binario 
		rec = rec+1
		wmf.models.write_int_basin(ruta_out_slides, R[c]['Slides_Map'],rec,cu.ncells,1)
f.close()

In [102]:
Qhist=True;Shist=True

In [ ]:
# def model_write_rutesHists(rutaConfig,Qhist=True,Shist=True):
#     ''' #Genera archivos vacios para cada parametrizacion cuando no existe historia o si esta quiere renovarse. 
#         Si se quiere dejar de crear rutas para alguno de los dos, se debe indicar False. e.g. Shist=False.
#         Genera un dataframe con la primera fila de un qsim, ssim.hdr cualquiera, resultado de simulacion de la cuenca de interes; 
#         la ruta de este archivo debe estar indicado en el configfile.
#         #Funcion no operacinal
#         #Argumentos:
#         -rutaConfig: string, ruta del configfile.
#         -Qhist: boolean, crear un Qhist. Default= True.
#         -Shist: boolean, crear un Shist. Default= True.
#     '''

ListConfig = al.get_rutesList(rutaConfig)

#Actualiza Qhist
if Qhist:
    #se lee la ruta donde se va a escribir
    ruta_qhist = al.get_ruta(ListConfig,'ruta_qsim_hist')
    #se lee el archivo que se va a copiar
    ruta_qsim = al.get_ruta(ListConfig,'ruta_qsim')
    rutas_qsim=glob.glob(ruta_qsim+'*.json')
    rutaqsim=rutas_qsim[0]
    #se leen los archivos de todas las par para crear Qhist para cada una.
    listrutas_qsim=np.sort(rutas_qsim)

    for i in listrutas_qsim:
        #Se lee el archivo Qsim de donde tomar la primera fila.
        qsim=pd.read_json(rutaqsim)
        Qh=qsim.iloc[[0]]
        nameQhistfile=i.split('/')[-1].split('.')[0]+'hist.'+i.split('/')[-1].split('.')[-1]
        #Pregunta si ya existe y si se quiere sobreescribir
        try:
            Lol = os.listdir(ruta_qhist)
            pos = Lol.index(nameQhistfile)
            flag = raw_input('Aviso: El archivo Qhist: '+nameQhistfile+' ya existe, desea sobre-escribirlo, perdera la historia de este!! (S o N): ')
            if flag == 'S':
                flag = True
            else:
                flag = False
        except:
            flag = True
        #Guardado
        if flag:
            Qh.to_json(ruta_qhist+nameQhistfile)
            print 'Aviso: Se crean los Qhist, el archivo Qsim usado para crear las rutas es: '+rutaqsim
        else:
            print 'Aviso: No se crean los Qhist'
else: 
    print ' Aviso: No se crean archivos Qhist.'

#Actualiza Shist
if Shist:
    ruta_shist = al.get_ruta(ListConfig,'ruta_almhist')
    ruta_ssim = al.get_ruta(ListConfig,'ruta_almsim')
    rutas_ssim=glob.glob(ruta_ssim+'*.StOhdr')
    rutassim=rutas_ssim[0]
    listrutas_ssim=np.sort(rutas_ssim)

    for j in listrutas_ssim:
        #Se lee el archivo Qsim de donde tomar la primera fila.
        ssim=pd.read_csv(j, header = 4, index_col = 5, parse_dates = True, usecols=(1,2,3,4,5,6))
        Sh=ssim.iloc[[0]]
        nameShistfile=j.split('/')[-1].split('.')[0]+'hist.'+i.split('/')[-1].split('.')[-1]
        #Pregunta si ya existe y si se quiere sobreescribir
        try:
            Lold = os.listdir(ruta_shist)
            pos = Lold.index(nameShistfile)
            flag = raw_input('Aviso: El archivo Shist: '+nameShistfile+' ya existe, desea sobre-escribirlo, perdera la historia de este!! (S o N): ')
            if flag == 'S':
                flag = True
            else:
                flag = False
        except:
            flag = True
        #Guardado
        if flag:
            Sh.to_json(ruta_shist+nameShistfile)
            print 'Aviso: Se crean los Shist , el archivo Ssim usado para crear las rutas es: '+rutassim
        else:
            print 'Aviso: No se crean los Shist'
else: 
    print ' Aviso: No se crean archivos S.'    